In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras


In [3]:
tf.__version__


'2.0.0'

In [21]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
train_data = pd.read_csv(TRAIN_DATA_URL)
test_data = pd.read_csv(TEST_DATA_URL)

In [7]:
print(train_data.shape)
print(test_data.shape)

(627, 10)
(264, 10)


In [8]:
train_data.describe()

,survived,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000,627.000000
mean,0.387560,29.631308,0.545455,0.379585,34.385399
std,0.487582,12.511818,1.151090,0.792999,54.597730
min,0.000000,0.750000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,0.000000,0.000000,7.895800
50%,0.000000,28.000000,0.000000,0.000000,15.045800
75%,1.000000,35.000000,1.000000,0.000000,31.387500
max,1.000000,80.000000,8.000000,5.000000,512.329200


In [9]:
train_data[train_data.isnull()].count()

survived              0
sex                   0
age                   0
n_siblings_spouses    0
parch                 0
fare                  0
class                 0
deck                  0
embark_town           0
alone                 0
dtype: int64

In [10]:
y_train= train_data.pop('survived')
y_test = test_data.pop('survived')


In [11]:
train_data.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [25]:
from sklearn.preprocessing import LabelEncoder
transfer = LabelEncoder()

cg_columns = ['sex','class','deck','embark_town','alone']

for col in cg_columns:
    train_data[col] = transfer.fit_transform(train_data[col] )
    test_data[col] = transfer.fit_transform(test_data[col])
    


In [57]:
train_data


,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,1,22.0,1,0,7.2500,2,7,2,0
1,1,0,38.0,1,0,71.2833,0,2,0,0
2,1,0,26.0,0,0,7.9250,2,7,2,1
3,1,0,35.0,1,0,53.1000,0,2,2,0
4,0,1,28.0,0,0,8.4583,2,7,1,1
...,...,...,...,...,...,...,...,...,...,...
622,0,1,28.0,0,0,10.5000,1,7,2,1
623,0,1,25.0,0,0,7.0500,2,7,2,1
624,1,0,19.0,0,0,30.0000,0,1,2,1
625,0,0,28.0,1,2,23.4500,2,7,2,0


In [32]:
train_dict = train_data.to_dict(orient="records")
test_dict = test_data.to_dict(orient="records")

In [31]:
# 4、字典特征抽取
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [33]:
transfer = DictVectorizer()
train_dict = transfer.fit_transform(train_dict)
test_dict = transfer.transform(test_dict)

In [37]:
train_data1 = pd.DataFrame(train_dict)

In [40]:
# 3）决策树预估器
estimator = DecisionTreeClassifier(criterion="entropy", max_depth=8)
estimator.fit(train_dict, y_train)

# 4）模型评估
# 方法1：直接比对真实值和预测值
y_predict = estimator.predict(test_dict)
print("y_predict:\n", y_predict)
print("直接比对真实值和预测值:\n", y_test == y_predict)

# 方法2：计算准确率
score = estimator.score(test_dict, y_test)
print("准确率为：\n", score)

# 可视化决策树
export_graphviz(estimator, out_file="data/titanic_tree.dot", feature_names=transfer.get_feature_names())

y_predict:
 [0 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0
 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 1
 0 1 0 1 1 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 1 1 1 0 1
 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0
 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0
 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1
 1 0 0 0 1]
直接比对真实值和预测值:
 0      True
1      True
2      True
3      True
4      True
       ... 
259    True
260    True
261    True
262    True
263    True
Name: survived, Length: 264, dtype: bool
准确率为：
 1.0


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

estimator = RandomForestClassifier()
# 加入网格搜索与交叉验证
# 参数准备
param_dict = {"n_estimators": [120,200,300,500,800,1200], "max_depth": [5,8,15,25,30]}
estimator = GridSearchCV(estimator, param_grid=param_dict, cv=3)
estimator.fit(train_data, y_train)

# 5）模型评估
# 方法1：直接比对真实值和预测值
y_predict = estimator.predict(test_data)
print("y_predict:\n", y_predict)
print("直接比对真实值和预测值:\n", y_test == y_predict)

# 方法2：计算准确率
score = estimator.score(test_data, y_test)
print("准确率为：\n", score)

# 最佳参数：best_params_
print("最佳参数：\n", estimator.best_params_)
# 最佳结果：best_score_
print("最佳结果：\n", estimator.best_score_)
# 最佳估计器：best_estimator_
print("最佳估计器:\n", estimator.best_estimator_)
# 交叉验证结果：cv_results_
print("交叉验证结果:\n", estimator.cv_results_)

y_predict:
 [0 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0
 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 1
 0 1 0 1 1 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 1 1 1 0 1
 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0
 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0
 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1
 1 0 0 0 1]
直接比对真实值和预测值:
 0      True
1      True
2      True
3      True
4      True
       ... 
259    True
260    True
261    True
262    True
263    True
Name: survived, Length: 264, dtype: bool
准确率为：
 1.0
最佳参数：
 {'max_depth': 5, 'n_estimators': 120}
最佳结果：
 1.0
最佳估计器:
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=Non

In [43]:
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()
estimator.fit(train_data,y_train)

/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [55]:
from sklearn.metrics import classification_report
y_predict = estimator.predict(test_data)
print(y_predict)
print('直接比对真实值和预测值\n', y_test == y_predict)
# 6.2计算准确率
score = estimator.score(test_data,y_test)
print('准确率评分为：',score)
#6.3 查看精确率、召回率、F1-score
report = classification_report(test_data,y_predict,labels=[2,4],target_names=['良性','恶性'])
print('精确率、召回率、F1-score报告：\n',report)
#6.4roc_auc_score
#6.4.1转化结果为0，1，用于二分类评估
# y_true = np.where(y_test>2,1,0)
# y_score = np.where(y_predict)
ra_score = roc_auc_score(y_test,y_score=y_predict)
print('roc_auc_score：',ra_score)

[0 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0
 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 1
 0 1 0 1 1 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 1 1 1 0 1
 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0
 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0
 1 0 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1
 1 0 0 0 1]
直接比对真实值和预测值
 0      True
1      True
2      True
3      True
4      True
       ... 
259    True
260    True
261    True
262    True
263    True
Name: survived, Length: 264, dtype: bool
准确率评分为： 1.0


ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

In [ ]:
categorocal_columns =['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
numeric_columns=['age','fare']
feature_columns = []
#处理离散值特征
for categorocal_column in categorocal_columns:
    vocab = train_data[categorocal_column].unique()
    print(categorocal_column,': ',vocab)
    feature_columns.append(
        tf.feature_column.indicator_column(
                    tf.feature_column.categorical_column_with_vocabulary_list(
                                                                              categorocal_column, vocab)))
# tf.feature_column.indicator_column,对离散数据进行one_hot编码

#处理连续型特征
for numeric_column in numeric_columns:
    feature_columns.append(
          tf.feature_column.numeric_column(
                      numeric_column, dtype=tf.float32))

In [ ]:
feature_columns[1]

In [ ]:
#构建datasets
def make_dataset(x,y,epochs=10,shuffle=True,batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((dict(x), y))#from_tensor_slices要传入字典
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset
    

In [ ]:
train_dataset = make_dataset(train_data, y_train,batch_size=30)

In [ ]:
for x,y in train_dataset.take(1):
    print(x, '\n\n', y, '\n\n',x['age'])


In [ ]:
#实验
# keras.layers.DenseFeatures可以将Feature定义的规则运用到dataset的每一个数据上

for x, y in train_dataset.take(1):
    age_column = feature_columns[7]
    gender_column = feature_columns[0]
    print(keras.layers.DenseFeatures(age_column)(x).numpy())#将age_column的规则运用到x上在打印numpy值
    print(keras.layers.DenseFeatures(gender_column)(x).numpy())
#出现warning是提示float64自动转为float32，tensorflow2可以忽略

In [ ]:
#实验
for x, y in train_dataset.take(1):
    print(keras.layers.DenseFeatures(feature_columns)(x).numpy())#将feature_columns的规则运用到x上在打印numpy值


In [ ]:
model = keras.models.Sequential([
    keras.layers.DenseFeatures(feature_columns),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(2, activation='softmax'),
])

model.compile(loss='sparse_categorical_crossentropy',
                     optimizer = keras.optimizers.SGD(lr=0.01),
                     metrics = ['accuracy'])



In [ ]:
# 1.model.fit
# 2.model -> estimator ->train
train_dataset = make_dataset(train_data, y_train,epochs=100)
test_dataset = make_dataset(test_data, y_test,epochs=1,shuffle=False)
history = model.fit(train_dataset,validation_split=0.2,steps_per_epoch=18,epochs=100)#steps_per_epoch=训练集的样本数//batch_size

In [ ]:
# 2.model -> estimator ->train
estimator = keras.estimator.model_to_estimator(model)
estimator.train(input_fn=lambda: make_dataset(train_data, y_train,epochs=100))
#input_fn：1、参数是一个函数
#            2、返回元组（features,labels）或（feature,label)